In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.model_selection import train_test_split
from datetime import timedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
!pip install optuna

In [ ]:
import optuna

## Import dataset
#### [StudentsPerformance dataset](https://www.kaggle.com/spscientist/students-performance-in-exams) as a benchmark

In [ ]:
dataset = pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')

In [ ]:
print(dataset.info())
print(dataset.describe())

## Split train and test data

In [ ]:
X = dataset.drop('writing score', axis='columns')
y = dataset['writing score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

## Preprocessing
#### There are no missing values. Just one hot encoding for categorical data.

In [ ]:
categorical_cols = [cname for cname in dataset.columns if dataset[cname].dtype == "object"]
print(categorical_cols)
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
    ])

## Hyperparameter Tuning

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

#### Define an objective function to be minimized.

In [ ]:
def objective(trial):

    # search better model from RandomForestRegressor, XGBRegressor
    regressor_name = trial.suggest_categorical('classifier', ['RandomForest', 'XGBoost'])
    # search better max_depth from 2 to 32
    max_depth = trial.suggest_int('max_depth', 2, 32)
    # search better n_estimators from 50 to 4000
    n_estimators = trial.suggest_int('n_estimators', 50, 4000)
    if regressor_name == 'RandomForest':
        regressor_obj = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=1234)
    else:
        regressor_obj = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=1234)

    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ("model", regressor_obj)
    ])
    
    error_list = cross_val_score(pipe, X_train, y_train, cv=3, scoring='neg_mean_squared_error')

    return error_list.mean()  # An objective value linked with the Trial object.

In [ ]:
study = optuna.create_study(direction='maximize')  # Create a new study.
study.optimize(objective, n_trials=50)  # Invoke optimization of the objective function.

In [ ]:
study.best_trial

## Best parameters

In [ ]:
print(f'classifier => {study.best_trial.params["classifier"]}')
print(f'max_depth => {study.best_trial.params["max_depth"]}')
print(f'n_estimators => {study.best_trial.params["n_estimators"]}')

## Compare to default parameter with test data

In [ ]:
default_model = RandomForestRegressor(random_state=1234)
default_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("model", default_model)
])
default_pipe.fit(X_train, y_train)
default_predict = default_pipe.predict(X_test)
default_score = mean_squared_error(y_test, default_predict)

In [ ]:
if study.best_trial.params["classifier"] == 'RandomForest':
    best_model = RandomForestRegressor(n_estimators=study.best_trial.params["n_estimators"], max_depth=study.best_trial.params["max_depth"], random_state=1234)
else:
    best_model = XGBRegressor(n_estimators=study.best_trial.params["n_estimators"], max_depth=study.best_trial.params["max_depth"], random_state=1234)
best_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ("model", best_model)
])
best_pipe.fit(X_train, y_train)
best_predict = best_pipe.predict(X_test)
best_score = mean_squared_error(y_test, best_predict)

In [ ]:
print(f'Score of dafault parameters => {default_score}')
print(f'Score of best parameters => {best_score}')